In [ ]:
!pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [33]:
import torch
import cv2
import numpy as np

def load_model():
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    return model

def detect_vehicle(model, image):
    results = model(image)
    results = results.pandas().xyxy[0]  # Results as DataFrame
    vehicles = results[results['name'].isin(['car'])]  # Filter for cars
    if not vehicles.empty:
        # Assuming one main vehicle per image for simplicity
        vehicle = vehicles.iloc[0]
        bbox = vehicle[['xmin', 'ymin', 'xmax', 'ymax']].astype(int).values.tolist()
        return bbox
    return None

def draw_dimension(image, bbox, dimension, font_scale=1, font_thickness=2):
    x1, y1, x2, y2 = bbox
    cv2.putText(image, f"{dimension:.2f}m", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), font_thickness)
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

def save_annotated_image(image, bbox, dimension, output_path):
    image_annotated = image.copy()
    draw_dimension(image_annotated, bbox, dimension)
    cv2.imwrite(output_path, image_annotated)

def estimate_dimensions(bbox_front, bbox_side, camera_params):
    # Unpack camera parameters
    f_front, cx_front, cy_front, D_front = camera_params['front']
    f_side, cx_side, cy_side, D_side = camera_params['side']

    # Extract bounding box dimensions
    width_px_front = bbox_front[2] - bbox_front[0]
    length_px_side = bbox_side[2] - bbox_side[0]

    # Simplified dimension estimation
    width = (width_px_front / cx_front) * D_front / f_front
    length = (length_px_side / cx_side) * D_side / f_side

    return width, length

# Example usage
model = load_model()
side_image_path = '/content/download (4).jpg'
front_image_path = '/content/download (5).jpg'

output_front_annotated = 'front_annotated.jpg'
output_side_annotated = 'side_annotated.jpg'

side_image = cv2.imread(side_image_path)
front_image = cv2.imread(front_image_path)

bbox_front = detect_vehicle(model, front_image)
bbox_side = detect_vehicle(model, side_image)

# Example camera parameters (focal length, optical center x, optical center y, distance to vehicle)
camera_params = {
    'front': (1000, 960, 540, 10000),  # Example values
    'side': (1000, 960, 540, 18000)  # Example values
}

if bbox_front and bbox_side:
    # Assume dimensions based on camera parameters
    dimensions = estimate_dimensions(bbox_front, bbox_side, camera_params)
    print(f"Estimated dimensions (width x length in meters): {dimensions}")
    # Save annotated images with dimensions
    save_annotated_image(front_image, bbox_front, dimensions[0], output_front_annotated)
    save_annotated_image(side_image, bbox_side, dimensions[1], output_side_annotated)

    print(f"Annotated images saved at {output_front_annotated} and {output_side_annotated}")
else:
    print("Vehicle not detected in one of the images.")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-4-14 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Estimated dimensions (width x length in meters): (1.5, 4.7625)
Annotated images saved at front_annotated.jpg and side_annotated.jpg
